# Apriori

Work is progress, compare to apyori for QA

In [152]:
import numpy as np
from apyori import apriori

In [153]:
items = np.arange(2,50)

names = []
for i in items:
    name = "".join([chr(i) for i in np.random.randint(65,91,size=4)])
    names.append(name)

In [154]:
def factorize(num):
    factors = []
    while(num>1):
        for i in range(2,num+1):
            if (num%i==0):
                factors.append(i)
                num = num//i
                break
    return factors

def compare_factores(a,b):
    a = a.copy()
    b = b.copy()
    before = len(a) + len(b)
    after = 0
    while (len(a)>0):
        i = a.pop()
        f = -1
        for idx in range(len(b)):
            j = b[idx]
            if (i==j):
                f = idx
                break
        if (f>-1):
            b.pop(f)
        else:
            after+=1
    after += len(a) + len(b)
    return 1-after/before

def similarity(a,b,minimum=0.05):
    sim = compare_factores(factorize(a),factorize(b))
    sim = (sim+minimum)/(1+minimum)
    return sim

In [235]:
def generate_purchase():
    # all items purchased by user
    purch = []

    # inflate 
    inflate = np.random.random() + 0.25
    
    # start with one random item 0-47
    start = np.random.randint(len(items))
    purch.append(start)
    shuffled = np.arange(len(items))
    np.random.shuffle(shuffled)
    for idx in shuffled:
        i = items[idx]
        chance = 0
        counter = 0
        for j in purch:
            chance+= similarity(i,j)
            counter+=1
        chance/=counter
        if (np.random.random()*inflate < chance):
            purch.append(idx)
    # convert to names
    purch = [names[i] for i in purch]
    return(purch)  

In [236]:
people = []
for i in range(1000):
    people.append(generate_purchase())
    
aa = np.array([len(x) for x in people])
aa.max(),aa.min(),aa.mean()

(44, 1, 18.705)

In [336]:
rules = apriori(people,min_support=0.03,min_confidence=0.2,min_lift=1.5,max_length=2)
a = list(rules)

In [337]:
# all supports
[i[1] for i in a]

[0.04, 0.066, 0.038, 0.031, 0.033, 0.073, 0.051, 0.039, 0.044, 0.056, 0.059]

In [338]:
# confidence
[i[2][0][2] for i in a]

[0.23529411764705882,
 0.5076923076923077,
 0.20994475138121546,
 0.23484848484848483,
 0.22758620689655176,
 0.6403508771929824,
 0.21794871794871792,
 0.3679245283018868,
 0.32352941176470584,
 0.2679425837320574,
 0.5566037735849056]

In [339]:
# lift
[i[2][0][3] for i in a]

[1.5582391897156214,
 1.5246015246015245,
 1.9806108620869383,
 2.060074428495481,
 1.67342799188641,
 1.6763111968402682,
 1.6025641025641022,
 1.5723270440251573,
 1.547987616099071,
 1.8737243617626396,
 1.7339681420090518]

In [340]:
# if they bought left, they will buy right using the confidence and lift as above (0th element)
a[0][2][0][0],a[0][2][0][1]

(frozenset({'DARN'}), frozenset({'QRGZ'}))

In [391]:
i = 1
a[i][2][0][0],a[i][2][0][1]

(frozenset({'MQQQ'}), frozenset({'FQEK'}))

In [349]:
users = len(people)
cont_m = np.array([np.sum([n in i for i in people]) for n in names])
supports = cont_m/users

In [344]:
confidence = np.zeros((len(names),len(names)))

In [347]:
for i in range(len(names)):
    for j in range(len(names)):
        for p in people:
            if names[i] in p and names[j] in p:
                confidence[i,j]+=1

In [356]:
confidence = confidence / cont_m

In [385]:
lift = confidence.T / supports

In [386]:
lift[0]

array([1.83823529, 1.08232545, 1.28019958, 1.09979889, 1.23584062,
       1.11283303, 1.27978407, 1.08232545, 1.21430872, 1.05968858,
       1.21948289, 1.13215816, 1.24270214, 1.1515382 , 1.23012343,
       1.16241349, 1.24321881, 1.06490872, 1.21460662, 1.10092114,
       1.28529998, 1.07980255, 1.22139841, 1.12580918, 1.22179896,
       1.20860352, 1.26462538, 1.2254902 , 1.1903581 , 1.00142669,
       1.24475891, 1.04040883, 1.28187778, 1.09879478, 1.25691302,
       0.96153846, 1.23413781, 0.98812047, 1.22437205, 0.98874777,
       1.1921069 , 1.1126161 , 1.2254902 , 1.11160302, 1.2254902 ,
       1.12721976, 1.21652866, 1.0460676 ])

In [387]:
np.argwhere(np.array(names) == 'DARN')[0][0]

9

In [388]:
np.argwhere(np.array(names) == 'QRGZ')[0][0]

11

In [389]:
confidence[9,11], confidence[11,9]

(0.26490066225165565, 0.23529411764705882)

In [390]:
lift[11,9]

1.5582391897156214

In [392]:
np.argwhere(np.array(names) == 'MQQQ')[0][0],np.argwhere(np.array(names) == 'FQEK')[0][0]

(35, 37)

In [394]:
confidence[37,35], lift[37,35]

(0.5076923076923077, 1.5246015246015245)